# Donor-Funded Project Success Prediction in Kenya

This notebook builds a predictive system using machine learning models to forecast the success of donor-funded projects in Kenya.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
# Data loading
# Load the dataset with a specified encoding
df = pd.read_csv("G:\My Drive\MSc. DSA\Module V\DSA 8502 Predictive & Optimisation Analytics\End of Module Project\donor-funded-projects-in-kenya-per-county-in-line-with-mdgs.csv", encoding='ISO-8859-1')

# Display the first few rows
df.head()


,Unspecified,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Project,Project Title,Total Project Cost (KES),Funding Type,Funding Source,Start Date (Planned),Start Date (Actual),Duration (Months),Vision 2030 Flagship Ministry,MTEF Sector,Implementing Agency,Implementation Status
2,2010/051489,Construction Of Kiambu West District Headquarters,"18,300,000",GoK; Unspecified,Government of Kenya,09/01/2009,09/01/2009,33 months,Unspecified,"Governance, Justice, Law and Order (GJLOS)",102 - Ministry of Interior and Coordination of...,Ongoing
3,2010/051491,Construction Of Taveta District Headquarters,"45,024,924",GoK; Unspecified,Government of Kenya,06/08/2008,06/02/2008,48 months,Unspecified,"Governance, Justice, Law and Order (GJLOS)",102 - Ministry of Interior and Coordination of...,Ongoing
4,2010/051501,Construction Of Kyuso District Headquarters,"50,000,000",GoK; Unspecified,Government of Kenya,03/02/2008,04/02/2008,51 months,Unspecified,"Governance, Justice, Law and Order (GJLOS)",102 - Ministry of Interior and Coordination of...,Ongoing


In [10]:
# Get the shape of the dataframe
rows, cols = df.shape
print(f'rows {rows}\ncols {cols}')

df.columns

rows 6351
cols 13


Index(['Project', 'Project Title', 'Total Project Cost (KES)', 'Funding Type',
       'Funding Source', 'Start Date (Planned)', 'Start Date (Actual)',
       'Duration (Months)', 'Vision 2030 Flagship Ministry', 'MTEF Sector',
       'Implementing Agency', 'Implementation Status', 'Success'],
      dtype='object')

In [12]:
# Data cleaning and preprocessing 

# Rename columns using the second row
df.columns = df.iloc[1]
df = df[2:].reset_index(drop=True)

# Drop any remaining unnamed columns with mostly missing data
df = df.dropna(axis=1, how='all')

# Rename columns to match expected names
df.columns = [
    "Project", "Project Title", "Total Project Cost (KES)", "Funding Type", 
    "Funding Source", "Start Date (Planned)", "Start Date (Actual)", 
    "Duration (Months)", "Vision 2030 Flagship Ministry", "MTEF Sector",
    "Implementing Agency", "Implementation Status", "Success"
]

# Keep relevant columns
columns_to_keep = [
    "Project", "Project Title", "Total Project Cost (KES)", "Funding Type", 
    "Funding Source", "Start Date (Planned)", "Start Date (Actual)", 
    "Duration (Months)", "Vision 2030 Flagship Ministry", "MTEF Sector",
    "Implementing Agency", "Implementation Status"
]
df = df[columns_to_keep]

# Drop rows with missing critical data
df = df.dropna(subset=["Project Title", "Total Project Cost (KES)", "Implementation Status"])

# Clean cost column
df["Total Project Cost (KES)"] = df["Total Project Cost (KES)"].replace({',': ''}, regex=True)

# Remove non-numeric rows (e.g., headers or invalid data)
df = df[pd.to_numeric(df["Total Project Cost (KES)"], errors="coerce").notnull()]

# Convert to float
df["Total Project Cost (KES)"] = df["Total Project Cost (KES)"].astype(float)

# Convert start dates
df["Start Date (Planned)"] = pd.to_datetime(df["Start Date (Planned)"], errors="coerce")
df["Start Date (Actual)"] = pd.to_datetime(df["Start Date (Actual)"], errors="coerce")

# Clean duration
df["Duration (Months)"] = df["Duration (Months)"].astype(str).str.extract(r'(\d+)').astype(float)

# Create binary target label
df["Success"] = df["Implementation Status"].apply(lambda x: 1 if "Completed" in str(x) else 0)

# Drop unused columns
df_model = df.drop(columns=["Project", "Project Title", "Start Date (Planned)", "Start Date (Actual)", "Implementation Status"])

# One-hot encoding
df_model = pd.get_dummies(df_model, drop_first=True)

# Preview processed data
df_model.head()


,Total Project Cost (KES),Duration (Months),Success,Funding Type_GoK; Unspecified,Funding Type_Grant,Funding Type_Grant; GoK,Funding Type_Grant; GoK; Unspecified,Funding Type_Grant; Unspecified,Funding Type_Loan,Funding Type_Loan ; GoK,...,Implementing Agency_MoHEST - NEP Technical Training Institute,Implementing Agency_MoHEST - Nkabune Technical Training Institute,Implementing Agency_MoHEST - Nyeri Technical Training Institute,Implementing Agency_MoHEST - Ramogi Institute of Advanced Technology,Implementing Agency_MoHEST - Rift Valley Institute of Science and Technology,Implementing Agency_MoHEST - Rift Valley Technical Training Institute,Implementing Agency_MoHEST - Wote Technical Training Institute,Implementing Agency_POPULATION SERVICES INTERNATIONAL (P.S.I),Implementing Agency_Private Sector,Implementing Agency_Unspecified
0,65600000.0,46.0,0,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,35000000.0,40.0,0,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,25000000.0,40.0,0,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,25000000.0,38.0,0,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,37500000.0,37.0,0,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [18]:
# Standardize column names
df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]

# Confirm column names
print("Columns:", df.columns.tolist())

# Handle missing values in key fields
df = df.dropna(subset=["project", "implementation_status"])

# Clean total project cost
df["total_project_cost_(kes)"] = (
    df["total_project_cost_(kes)"]
    .astype(str)  # Ensure the column is treated as strings
    .str.replace(",", "", regex=False)
    .str.extract(r"(\d+)", expand=False)
    .astype(float)
)

# Clean duration to numeric
df["duration_(months)"] = (
    df["duration_(months)"]
    .astype(str)  # Ensure the column is treated as strings
    .str.extract(r"(\d+)", expand=False)
    .astype(float)
)

# Fill missing actual start dates with planned dates
df["start_date_(actual)"].fillna(df["start_date_(planned)"], inplace=True)

# Handle missing categorical fields
categorical_cols = [
    "funding_type", "funding_source",
    "vision_2030_flagship_ministry", "mtef_sector", "implementing_agency"
]
df[categorical_cols] = df[categorical_cols].fillna("Unknown")

# Remove duplicates
df = df.drop_duplicates()

# Standardize categorical values (lowercase, no extra spaces)
df[categorical_cols] = df[categorical_cols].apply(lambda x: x.str.lower().str.strip())

# Show info and remaining nulls
df.info()
print("\nMissing values:\n", df.isnull().sum())

df.head()


Columns: ['project', 'project_title', 'total_project_cost_(kes)', 'funding_type', 'funding_source', 'start_date_(planned)', 'start_date_(actual)', 'duration_(months)', 'vision_2030_flagship_ministry', 'mtef_sector', 'implementing_agency', 'implementation_status', 'success']
<class 'pandas.core.frame.DataFrame'>
Index: 2840 entries, 0 to 6346
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   project                        2840 non-null   object        
 1   project_title                  2840 non-null   object        
 2   total_project_cost_(kes)       2840 non-null   float64       
 3   funding_type                   2840 non-null   object        
 4   funding_source                 2840 non-null   object        
 5   start_date_(planned)           1946 non-null   datetime64[ns]
 6   start_date_(actual)            2105 non-null   datetime64[ns]
 7   duration_(month

C:\Users\Joyce\AppData\Local\Temp\ipykernel_27828\3171393661.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["start_date_(actual)"].fillna(df["start_date_(planned)"], inplace=True)


,project,project_title,total_project_cost_(kes),funding_type,funding_source,start_date_(planned),start_date_(actual),duration_(months),vision_2030_flagship_ministry,mtef_sector,implementing_agency,implementation_status,success
0,2010/051503,Construction Of Nzaui District Headquarters,65600000.0,gok; unspecified,government of kenya,2008-08-03,2008-04-03,46.0,unspecified,"governance, justice, law and order (gjlos)",102 - ministry of interior and coordination of...,Ongoing,0
1,2010/051509,Construction Of Kibwezi District Headquarters,35000000.0,gok; unspecified,government of kenya,2009-02-03,2009-02-03,40.0,unspecified,"governance, justice, law and order (gjlos)",102 - ministry of interior and coordination of...,Ongoing,0
2,2010/051517,Construction Of Meru Central District Headquar...,25000000.0,gok; unspecified,government of kenya,2009-02-04,2009-02-04,40.0,unspecified,"governance, justice, law and order (gjlos)",102 - ministry of interior and coordination of...,Ongoing,0
3,2010/051518,Construction Of Imenti South District Headquar...,25000000.0,gok; unspecified,government of kenya,NaT,2009-05-01,38.0,unspecified,"governance, justice, law and order (gjlos)",102 - ministry of interior and coordination of...,Ongoing,0
4,2010/051519,Construction Of Lagdera District Headquarters,37500000.0,gok; unspecified,government of kenya,2009-05-01,2009-06-01,37.0,unspecified,"governance, justice, law and order (gjlos)",102 - ministry of interior and coordination of...,Ongoing,0


In [42]:
%pip install plotly


Note: you may need to restart the kernel to use updated packages.


In [55]:
import plotly.express as px

# Extract year from start dates
df["Start_Year"] = df["start_date_(actual)"].dt.year

# Count projects per year
projects_per_year = df.groupby("Start_Year").size().reset_index(name="Number of Projects")

# Create an interactive line plot for projects started over time with a modern color palette
fig = px.line(projects_per_year, 
              x="Start_Year", 
              y="Number of Projects", 
              markers=True,
              title="Trend of Donor-Funded Projects Over Time",
              labels={"Start_Year": "Year", "Number of Projects": "Number of Projects"},
              line_shape="linear",
              color_discrete_sequence=["#636EFA"])  # Modern blue color

fig.update_layout(
    xaxis_title="Year", 
    yaxis_title="Number of Projects", 
    template="plotly",  # Use a clean modern theme
    title_x=0.5,  # Center the title
    plot_bgcolor="white",  # Set plot background to white
    paper_bgcolor="white"  # Set the overall paper background to white
)
fig.show()



In [56]:
# Investigating project success based on implementation status
success_counts = df["implementation_status"].value_counts().reset_index(name="Number of Projects")
success_counts.columns = ["Implementation Status", "Number of Projects"]

# Create an interactive bar plot for project success distribution with modern color palette
fig = px.bar(success_counts, 
             x="Implementation Status", 
             y="Number of Projects", 
             title="Project Success Distribution",
             labels={"Implementation Status": "Implementation Status", "Number of Projects": "Number of Projects"},
             color="Implementation Status",  # Color bars by Implementation Status
             color_discrete_sequence=["#F39C12", "#8E44AD", "#1ABC9C", "#E74C3C"])  # Vibrant modern colors
fig.update_layout(
    xaxis_title="Implementation Status", 
    yaxis_title="Number of Projects", 
    template="plotly",  # Use a clean modern theme
    title_x=0.5,  # Center the title
    plot_bgcolor="white",  # Set plot background to white
    paper_bgcolor="white"  # Set the overall paper background to white
)
fig.show()




In [57]:
import plotly.express as px
import pandas as pd

# Grouping and normalizing data for funding type vs implementation status
funding_vs_success = df.groupby(["funding_type", "implementation_status"]).size().unstack()
funding_vs_success_percentage = funding_vs_success.div(funding_vs_success.sum(axis=1), axis=0) * 100

# Create an interactive heatmap for funding type vs implementation status
fig = px.imshow(funding_vs_success_percentage.T,  # Transpose for correct orientation
                labels=dict(x="Funding Type", y="Implementation Status", color="% Projects"),
                x=funding_vs_success_percentage.index,
                y=funding_vs_success_percentage.columns,
                color_continuous_scale="Blues",  # Modern blue color scale
                title="Funding Type vs. Project Implementation Status (%)")
fig.update_layout(
    template="plotly",  # Use modern plotly template
    title_x=0.5,  # Center the title
    plot_bgcolor="white",  # White plot background
    paper_bgcolor="white",  # White paper background
)
fig.show()



In [59]:
# Grouping and normalizing data for MTEF sector vs implementation status
sector_vs_success = df.groupby(["mtef_sector", "implementation_status"]).size().unstack()
sector_vs_success_percentage = sector_vs_success.div(sector_vs_success.sum(axis=1), axis=0) * 100

# Create an interactive heatmap for MTEF sector vs implementation status
fig = px.imshow(sector_vs_success_percentage.T,  # Transpose for correct orientation
                labels=dict(x="MTEF Sector", y="Implementation Status", color="% Projects"),
                x=sector_vs_success_percentage.index,
                y=sector_vs_success_percentage.columns,
                color_continuous_scale="Greens",  # Modern green color scale
                title="MTEF Sector vs. Project Implementation Status (%)")
fig.update_layout(
    template="plotly",  # Use modern plotly template
    title_x=0.5,  # Center the title
    plot_bgcolor="white",  # White plot background
    paper_bgcolor="white",  # White paper background
)
fig.show()


In [19]:
# Split data
X = df_model.drop("Success", axis=1)
y = df_model["Success"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n{name} Report:")
    print(classification_report(y_test, y_pred))
    results[name] = {
        "accuracy": accuracy_score(y_test, y_pred)
    }





Logistic Regression Report:
              precision    recall  f1-score   support

           0       0.86      1.00      0.92      1087
           1       0.00      0.00      0.00       183

    accuracy                           0.86      1270
   macro avg       0.43      0.50      0.46      1270
weighted avg       0.73      0.86      0.79      1270


Decision Tree Report:
              precision    recall  f1-score   support

           0       0.95      0.96      0.96      1087
           1       0.77      0.73      0.75       183

    accuracy                           0.93      1270
   macro avg       0.86      0.84      0.85      1270
weighted avg       0.93      0.93      0.93      1270



C:\Users\Joyce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Joyce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Joyce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati


Random Forest Report:
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      1087
           1       0.84      0.67      0.75       183

    accuracy                           0.93      1270
   macro avg       0.89      0.83      0.85      1270
weighted avg       0.93      0.93      0.93      1270



C:\Users\Joyce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [12:59:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



XGBoost Report:
              precision    recall  f1-score   support

           0       0.93      0.98      0.96      1087
           1       0.84      0.57      0.68       183

    accuracy                           0.92      1270
   macro avg       0.89      0.78      0.82      1270
weighted avg       0.92      0.92      0.92      1270



In [30]:
import plotly.express as px
import pandas as pd
from sklearn.metrics import accuracy_score

# Calculate model accuracy scores
model_scores = {
    "Logistic Regression": accuracy_score(y_test, models["Logistic Regression"].predict(X_test)),
    "Decision Tree": accuracy_score(y_test, models["Decision Tree"].predict(X_test)),
    "Random Forest": accuracy_score(y_test, models["Random Forest"].predict(X_test)),
    "XGBoost": accuracy_score(y_test, models["XGBoost"].predict(X_test)),
}

# Convert to percentage and DataFrame
score_df = pd.DataFrame(
    [(model, round(score * 100, 2)) for model, score in model_scores.items()],
    columns=["Model", "Accuracy (%)"]
).sort_values(by="Accuracy (%)", ascending=True)

# Create interactive bar chart
fig = px.bar(
    score_df,
    x="Accuracy (%)",
    y="Model",
    orientation='h',
    text=score_df["Accuracy (%)"].apply(lambda x: f"{x:.2f}%"),
    title="Accuracy Comparison of ML Models",
    color="Model",
    color_discrete_sequence=px.colors.sequential.Viridis
)

# Customize layout
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_range=[0, 105],
    xaxis_title="Accuracy (%)",
    yaxis_title=None,
    title_font_size=18,
    template="plotly_white",
    height=450
)

# Display the interactive chart
fig.show()

